# Notebook for scrapping the Datasets from the Website

In [4]:
#importing the libraries
import pytesseract
import cv2
import numpy as np
import pandas as pd
import requests
import re
import os
import pdfplumber
import fitz
import camelot

from PIL import Image
from bs4 import BeautifulSoup
from pdf2image import convert_from_path

C:\Users\Administrator\anaconda3\envs\learn-env\Lib\site-packages\pypdf\_crypt_providers\_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


In [5]:

# List of URLs to scrape
urls = [
    "https://www.knbs.or.ke/cpi-and-inflation-rates/",
    "https://www.knbs.or.ke/cpi-and-inflation-rates/page/2/",
    "https://www.knbs.or.ke/cpi-and-inflation-rates/page/3/",
    "https://www.knbs.or.ke/cpi-and-inflation-rates/page/4/",
    "https://www.knbs.or.ke/cpi-and-inflation-rates/page/5/",
    "https://www.knbs.or.ke/cpi-and-inflation-rates/page/6/",
]

# Folder to save documents
os.makedirs("KNBS_dataset", exist_ok=True)

# Function to download PDFs
def download_pdf(pdf_url, base_url):
    if not pdf_url.startswith("http"):
        pdf_url = base_url + pdf_url  # Handle relative links
    
    filename = os.path.join("KNBS_dataset", pdf_url.split("/")[-1])
    print(f"📥 Downloading: {pdf_url}")

    try:
        pdf_response = requests.get(pdf_url, headers={"User-Agent": "Mozilla/5.0"}, verify=False)
        if pdf_response.status_code == 200:
            with open(filename, "wb") as f:
                f.write(pdf_response.content)
            print(f"✅ Saved: {filename}")
        else:
            print(f"❌ Failed to download: {pdf_url} (Status: {pdf_response.status_code})")
    except Exception as e:
        print(f"❌ Error downloading {pdf_url}: {e}")

# Function to scrape text content from articles
def scrape_text(url):
    try:
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, verify=False)
        soup = BeautifulSoup(response.text, "html.parser")
        
        text_content = [p.get_text() for p in soup.find_all("p")]

        # Save text to a file
        filename = os.path.join("KNBS_dataset", url.split("/")[-2] + ".txt")
        with open(filename, "w", encoding="utf-8") as f:
            f.write("\n".join(text_content))      
        
        print(f"✅ Saved article: {filename}")
    except Exception as e:
        print(f"❌ Error scraping {url}: {e}")

# Compile regex pattern for filtering links
keyword_pattern = re.compile(r"consumer price indices|inflation rates", re.IGNORECASE)

# Scraping all URLs
for url in urls:
    print(f"🔍 Scraping: {url}")
    base_url = "/".join(url.split("/")[:3])  # Extract base URL (e.g., "https://www.knbs.or.ke")

    try:
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, verify=False)
        soup = BeautifulSoup(response.text, "html.parser")

        # Find all links on the page
        for link in soup.find_all("a", href=True):
            href = link["href"].strip()
            text = link.get_text(strip=True)

            # Check for PDF links
            if href.endswith(".pdf"):
                full_link = href if href.startswith("http") else f"{base_url}{href}"
                download_pdf(full_link, base_url)

        # Scrape and save article text
        scrape_text(url)
    
    except Exception as e:
        print(f"❌ Error accessing {url}: {e}")

print("✅ Scraping Complete!")


🔍 Scraping: https://www.knbs.or.ke/cpi-and-inflation-rates/


C:\Users\Administrator\anaconda3\envs\learn-env\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.knbs.or.ke'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


📥 Downloading: https://www.knbs.or.ke/wp-content/uploads/2024/06/Kenya-Consumer-Price-Indices-Article-on-CPI.pdf


C:\Users\Administrator\anaconda3\envs\learn-env\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.knbs.or.ke'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


✅ Saved: KNBS_dataset\Kenya-Consumer-Price-Indices-Article-on-CPI.pdf
📥 Downloading: https://www.knbs.or.ke/wp-content/uploads/2025/01/Core-Non-Core-Inflation-DG-Presentation.pdf


...

✅ Saved: KNBS_dataset\Kenya-Consumer-Price-Indices-and-Inflation-Rates-December-2014.pdf
📥 Downloading: https://www.knbs.or.ke/wp-content/uploads/2023/09/Kenya-Consumer-Price-Indices-and-Inflation-Rates-October-2014.pdf
✅ Saved: KNBS_dataset\Kenya-Consumer-Price-Indices-and-Inflation-Rates-October-2014.pdf
📥 Downloading: https://www.knbs.or.ke/wp-content/uploads/2023/09/Kenya-Consumer-Price-Indices-and-Inflation-Rates-September-2014.pdf


C:\Users\Administrator\anaconda3\envs\learn-env\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.knbs.or.ke'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\Administrator\anaconda3\envs\learn-env\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.knbs.or.ke'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


✅ Saved: KNBS_dataset\Kenya-Consumer-Price-Indices-and-Inflation-Rates-September-2014.pdf
📥 Downloading: https://www.knbs.or.ke/wp-content/uploads/2023/09/Kenya-Consumer-Price-Indices-and-Inflation-Rates-August-2014.pdf
✅ Saved: KNBS_dataset\Kenya-Consumer-Price-Indices-and-Inflation-Rates-August-2014.pdf


C:\Users\Administrator\anaconda3\envs\learn-env\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.knbs.or.ke'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\Administrator\anaconda3\envs\learn-env\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.knbs.or.ke'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


✅ Saved article: KNBS_dataset\6.txt
✅ Scraping Complete!


In [29]:

# Define folder containing PDFs
pdf_folder = "KNBS_dataset" 
output_base_folder = "Extracted_CSVs"

# Function to extract year and month from filename
def extract_year_month(filename):
    year_match = re.search(r"(19|20)\d{2}", filename) 
    month_match = re.search(r"(January|February|March|April|May|June|July|August|September|October|November|December)", filename, re.IGNORECASE)
    
    year = year_match.group(0) if year_match else "unknown_year"
    month = month_match.group(0).capitalize() if month_match else "unknown_month"
    
    return year, month

# Loop through all PDFs in the folder
for filename in os.listdir(pdf_folder):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(pdf_folder, filename)
        year, month = extract_year_month(filename)  

        # Create year/month folder structure
        output_folder = os.path.join(output_base_folder, year, month)
        os.makedirs(output_folder, exist_ok=True)

        # Pre-create CSV files with headers 
        csv_paths = {
            "cpi": os.path.join(output_folder, "cpi_inflation_rates.csv"),
            "overall_cpi": os.path.join(output_folder, "overall_cpi_trends.csv"),
            "prices": os.path.join(output_folder, "national_average_prices.csv")
        }

        for csv_file in csv_paths.values():
            if not os.path.exists(csv_file): 
                pd.DataFrame().to_csv(csv_file, index=False)

        # Initialize lists for different tables
        cpi_data, overall_cpi_data, prices_data = [], [], []

        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                tables = page.extract_tables()
                for table in tables:
                    df = pd.DataFrame(table)
                    
                    # Skip empty tables
                    if df.empty:
                        continue

                    # Assign correct headers if table structure is known
                    df.columns = [f"Column_{i}" for i in range(len(df.columns))]

                    # Identify the table type based on keywords in the first column
                    first_cell = str(df.iloc[0, 0]).lower() if df.iloc[0, 0] else ""

                    if "cpi" in first_cell and "inflation" in first_cell:
                        df["Year"] = year
                        df["Month"] = month
                        cpi_data.append(df)

                    elif "overall new cpi" in first_cell:
                        df["Year"] = year
                        df["Month"] = month
                        overall_cpi_data.append(df)

                    elif any(keyword in first_cell for keyword in ["commodity", "average price"]):
                        df["Year"] = year
                        df["Month"] = month
                        prices_data.append(df)

        # Save tables to year/month-specific CSVs, appending data if the file exists
        for category, data in zip(["cpi", "overall_cpi", "prices"], [cpi_data, overall_cpi_data, prices_data]):
            if data:
                final_df = pd.concat(data, ignore_index=True)
                
                # If the file exists, append data
                if os.path.exists(csv_paths[category]):
                    final_df.to_csv(csv_paths[category], mode='a', header=False, index=False)
                else:
                    final_df.to_csv(csv_paths[category], index=False)

print("✅ Extraction complete! CSVs are saved in 'Extracted_CSVs' folder by year and month.")

✅ Extraction complete! CSVs are saved in 'Extracted_CSVs' folder by year and month.
